In [71]:
from bs4 import BeautifulSoup
import requests, string, math, os, shutil

In [39]:
URL = "https://skybrary.aero"
webSite = requests.get(URL)
soup = BeautifulSoup(webSite.content, "html.parser")

# find all a tags in div with class="sf-depth-1 menuparent" or "sf-depth-1"
navigations = soup.find_all("a", class_=["sf-depth-1 menuparent", "sf-depth-1"])

dict_navigations={}

for navigation in navigations:
    dict_navigations[navigation.text.strip()] = URL + navigation["href"]

dict_navigations

{'Portals': 'https://skybrary.aero/portals',
 'Browsers': 'https://skybrary.aero/browsers',
 'Themes': 'https://skybrary.aero/themes-0',
 'Tutorials': 'https://skybrary.aero/tutorials',
 'Courses': 'https://skybrary.aero/courses',
 'Publications': 'https://skybrary.aero/publications'}

In [53]:
portals_page = requests.get(dict_navigations["Portals"])
portals_soup = BeautifulSoup(portals_page.content, "html.parser")

# find all a tags in div with class="term-name"
portal_categories = portals_soup.find_all("div", class_="term-name")

array_portal_categories=[]
for portal_category in portal_categories:
    # add link to array_portal_categories
    array_portal_categories.append([URL + portal_category.find("a")["href"], portal_category.text.strip()])

array_portal_categories

[['https://skybrary.aero/operational-issues/air-ground-communication',
  'Air Ground Communication'],
 ['https://skybrary.aero/operational-issues/airspace-infringement',
  'Airspace Infringement'],
 ['https://skybrary.aero/operational-issues/wildlife-strike',
  'Wildlife Strike'],
 ['https://skybrary.aero/operational-issues/controlled-flight-terrain',
  'Controlled Flight Into Terrain'],
 ['https://skybrary.aero/operational-issues/fire-smoke-and-fumes',
  'Fire Smoke and Fumes'],
 ['https://skybrary.aero/operational-issues/ground-operations',
  'Ground Operations'],
 ['https://skybrary.aero/operational-issues/airworthiness', 'Airworthiness'],
 ['https://skybrary.aero/operational-issues/level-bust', 'Level Bust'],
 ['https://skybrary.aero/operational-issues/loss-control', 'Loss of Control'],
 ['https://skybrary.aero/operational-issues/loss-separation',
  'Loss of Separation'],
 ['https://skybrary.aero/operational-issues/runway-excursion',
  'Runway Excursion'],
 ['https://skybrary.aero/

In [72]:
def ParserCategory(URL, name_category):
    
    print("PARSING: "+URL)
    category_page = requests.get(URL)
    category_soup = BeautifulSoup(category_page.content, "html.parser")
    
    div_articles = category_soup.find_all("div", class_="title")
    span_articles = category_soup.find_all("span", class_="field-content")
    articles = div_articles + span_articles

    for article in articles:
        
        URL_article = "https://skybrary.aero" + article.find("a")["href"]
        webSite_article = requests.get(URL_article)
        soup_article = BeautifulSoup(webSite_article.content, "html.parser")
        group_items = soup_article.find("div", class_="group-left-bottom")
        
        # Initialiser un dictionnaire pour stocker le contenu des <h2> et <p>
        data_dict = {}

        # Parcourir le contenu HTML pour récupérer le contenu des <h2> et <p>
        current_h2 = None
        current_p = []
        for tag in group_items.find_all(['h2', 'p', 'li']):
            if tag.name == 'h2':
                # Enregistrer le contenu du dernier <h2> et ses <p> associés
                if current_h2 is not None and current_p:
                    data_dict[current_h2] = current_p

                # Mettre à jour la valeur du dernier <h2>
                current_h2 = tag.text.strip()

                # Réinitialiser la liste des <p> associés au nouveau <h2>
                current_p = []
            elif tag.name == 'p':
                # Ajouter le contenu du <p> à la liste en cours
                current_p.append(tag.text.strip())

            elif tag.name == 'li':
                # Ajouter le contenu du <li> à la liste en cours
                current_p.append(tag.text.strip()+".")

        # Sauvegarder le dernier <h2> et ses <p> associés (s'il y en a)
        if current_h2 is not None and current_p:
            data_dict[current_h2] = current_p

        # Écrire les données dans des fichiers .txt
        for h2, p_content in data_dict.items():
            # Utiliser le nom du fichier basé sur la valeur de <h2> (sans caractères invalides)
            filename_prefix = f"{h2.translate(str.maketrans('', '', string.punctuation))}"

            # Limiter le contenu de <p> à 500 mots par fichier
            words_limit = 500
            total_words = len(" ".join(p_content).split())
            num_files = math.ceil(total_words / words_limit)

            for i in range(num_files):
                # Sélectionner les mots pour chaque fichier
                start_idx = i * words_limit
                end_idx = (i + 1) * words_limit
                words = " ".join(p_content).split()[start_idx:end_idx]

                # Construire le contenu du fichier
                file_content = " ".join(words)

                # Numéroter le fichier s'il y a plus d'un fichier
                if num_files > 1:
                    filename = f"/{filename_prefix}_{i+1}.txt"
                else:
                    filename = f"/{filename_prefix}.txt"

                # Si répertoire name_category n'existe pas alors le créer
                if os.path.exists("Skybrary_data/"+name_category) == False:
                    os.mkdir("Skybrary_data/"+name_category)

                # Si répertoire article.text n'existe pas alors le créer
                directory_name = f"{article.text.translate(str.maketrans('', '', string.punctuation))}"
                if os.path.exists("Skybrary_data/"+name_category+"/"+directory_name) == False:
                    os.mkdir("Skybrary_data/"+name_category+"/"+directory_name)

                # Écrire le contenu dans le fichier .txt
                with open("Skybrary_data/"+name_category +"/"+ directory_name + filename, 'w', encoding='utf-8') as file:
                    file.write(file_content)

if os.path.exists("Skybrary_data") == True:
    #supprimer le dossier data et son contenu
    shutil.rmtree("Skybrary_data")
# Créer le dossier data
os.mkdir("Skybrary_data")

for category in array_portal_categories:
    ParserCategory(category[0], category[1])

PARSING: https://skybrary.aero/operational-issues/air-ground-communication
PARSING: https://skybrary.aero/operational-issues/airspace-infringement
PARSING: https://skybrary.aero/operational-issues/wildlife-strike
PARSING: https://skybrary.aero/operational-issues/controlled-flight-terrain
PARSING: https://skybrary.aero/operational-issues/fire-smoke-and-fumes
PARSING: https://skybrary.aero/operational-issues/ground-operations
PARSING: https://skybrary.aero/operational-issues/airworthiness
PARSING: https://skybrary.aero/operational-issues/level-bust
PARSING: https://skybrary.aero/operational-issues/loss-control
PARSING: https://skybrary.aero/operational-issues/loss-separation
PARSING: https://skybrary.aero/operational-issues/runway-excursion
PARSING: https://skybrary.aero/operational-issues/runway-incursion
PARSING: https://skybrary.aero/operational-issues/wake-vortex-turbulence
PARSING: https://skybrary.aero/operational-issues/weather
PARSING: https://skybrary.aero/operational-issues/eme